In [168]:
import warnings
warnings.filterwarnings("ignore")
import os
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

# I/O files
# Note: before running, create a folder in output_location
# NoTE: before running, locate all the data files under folder_location 
output_location = r"C:\data\output"
folder_location = r"C:\data\input"
dir=Path(folder_location)
all_files = os.listdir(dir)
List, Pts =[],[]
for f in all_files:
    if f.endswith(".csv"):
        df = pd.read_csv(folder_location +"/" + f, sep=",")
        Pts.append(folder_location +"/" + f)
        List.append(df) 
        
# In each file, extract X, Y cell coordinates, run <dbscan>, output as an individual file   
for k in np.arange(0, len(Pts)):
    List[k] = List[k].rename(columns={"Cell X Position NEW":"Cell.X.Position","Cell Y Position NEW":"Cell.Y.Position",
                                     "Complex Phenotype":"Complex.Phenotype"})
    
    # extract for "target cells" (cells of interest) for clustering analysis
    ListTargetCells=List[k][~List[k].Phenotype.isin(["CD8+","Others"])]
    data = pd.DataFrame({"x": ListTargetCells["Cell.X.Position"], "y": ListTargetCells["Cell.Y.Position"]}).values.tolist()
    X = np.array(data)
    # dbscan
    db = DBSCAN(eps=70, min_samples=7).fit(X)
    labels = db.labels_
    
    # Format the results
    # Raw data plus cluster
    XY = pd.DataFrame(X, columns=["Cell.X.Position","Cell.Y.Position"])
    Cluster = pd.DataFrame(labels, columns =["Cluster"])
    XYCluster=pd.concat([XY, Cluster], axis=1)
    OutputFile = pd.merge(List[k],XYCluster, on=["Cell.X.Position","Cell.Y.Position"], how="outer")
    OutputFile.to_csv(output_location +"\\"+ Pts[k].split("/")[1])
    # Only the target-cell information
    OutputFile_ = OutputFile[["Path","Complex.Phenotype","Cell.X.Position",
                              "Cell.Y.Position","Cluster"]]
    OutputFile_.to_csv(output_location +"\\"+ "ClusterCells_" + Pts[k].split("/")[1])
    

# Put output files to one file 
file = []
dir=Path(output_location)
all_files = os.listdir(dir)
for f in all_files:
    if f.startswith("ClusterCells_"):
        file.append(pd.read_csv(f))
        Results = pd.concat(file)
Results.to_csv(output_location + "\\" + "Results_all.txt")
Results.head(5)